In [2]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import talib
import mpl_finance as mpf
%matplotlib inline
import seaborn as sns
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
import csv
import time
plt.rcParams['font.family']='SimHei' # 顯示中文('SimHei' for MacOS)
from sklearn import preprocessing

df1=pd.read_json('../Tw50.json')
df1.sort_index(inplace=True)
def RSI(n,index):
    mean_up=0
    mean_down=0
    data=df1.loc[index-(n-1):index+1,'漲跌價差']
    for i in data :
        if float(i)>0:
            mean_up+=float(i)
        else:
            mean_down-=float(i)
    mean_up/=n
    mean_down/=n
    if mean_down == 0 :
        return np.nan
    else :
        rs=mean_up/mean_down
        rsi=100*rs/(1+rs)
        return rsi
def KD(n,index,K_before,D_before):
    Mtoday=float(df1.at[index,'收盤價'])
    data=df1.loc[index-(n-1):index+1,'收盤價']
    Max=float(max(data))
    Min=float(min(data))
    RSV=(Mtoday-Min)/(Max-Min) *100
    K=K_before*0.6667+RSV*0.3333
    D=D_before*0.6667+K*0.3333
    return (K,D)
#RSI_6
N_day=6
rsi=RSI(N_day,N_day-1)
df1.loc[N_day-1,'RSI_6']=rsi
for i in range(N_day,len(df1)):
    rsi=RSI(N_day,i)
    df1.loc[i,'RSI_6']=rsi
#RSI_12
N_day=12
rsi=RSI(N_day,N_day-1)
df1.loc[N_day-1,'RSI_12']=rsi
for i in range(N_day,len(df1)):
    rsi=RSI(N_day,i)
    df1.loc[i,'RSI_12']=rsi

for i in range(len(df1)-1):
    dif=float(df1.loc[i+1,'最高價'])-float(df1.loc[i,'最高價'])
    if dif > 0:
        df1.loc[i,'Ans']=1
    else:
        df1.loc[i,'Ans']=-1
#KD_5
N_day=5
K,D=KD(N_day,N_day-1,50,50)
df1.loc[N_day-1,'K']=K
df1.loc[N_day-1,'D']=D
for i in range(N_day,len(df1)):
    K,D=KD(N_day,i,K,D)
    df1.loc[i,'K']=K
    df1.loc[i,'D']=D
for i in range(12,len(df1)):
    Yesterday=df1.loc[i-1,'RSI_12']-df1.loc[i-1,'RSI_6']
    Today    =df1.loc[i,'RSI_12']-df1.loc[i,'RSI_6']
    Product = Yesterday*Today
    if Product < 0 or (abs(Product-0)<0.0001) :
        if Yesterday > 0:
            df1.loc[i,'RSI_DIF']=1
        else :
            df1.loc[i,'RSI_DIF']=-1
    else :
        df1.loc[i,'RSI_DIF']=0
#BIAS
def BIAS(n,index):
    Mtoday=float(df1.at[index,'收盤價'])
    data=np.asarray(df1.loc[index-(n-1):index+1,'收盤價'].astype(float))
    Mean=np.mean(data)
    ans=100*(Mtoday-Mean)/Mean
    return ans
N_day=6
bias=BIAS(N_day,N_day-1)
df1.loc[N_day-1,'BIAS']=bias
for i in range(N_day,len(df1)):
    bias=BIAS(N_day,i)
    df1.loc[i,'BIAS']=bias
#WMR
def WMR(n,index):
    Mtoday=float(df1.at[index,'收盤價'])
    H=np.asarray(df1.loc[index-(n-1):index+1,'最高價'].astype(float))
    L=np.asarray(df1.loc[index-(n-1):index+1,'最低價'].astype(float))
    Hn=np.max(H)
    Ln=np.min(L)
    ans=100*(Hn-Mtoday)/(Hn-Ln)
    return ans
N_day=6
wmr=WMR(N_day,N_day-1)
df1.loc[N_day-1,'WMR']=wmr
for i in range(N_day,len(df1)):
    wmr=WMR(N_day,i)
    df1.loc[i,'WMR']=wmr
#MACD
def EMA(index,N):
    S='EMA_'+str(N)
    ans=((float(df1.at[index,'收盤價'])*2)+(N-1)*(float(df1.at[index-1,S])))/(N+1)
    return ans
def MACD(index,N,dif):
    ans=((dif*2)+(N-1)*(float(df1.at[index-1,'MACD'])))/(N+1)
    return ans
df1.loc[0,'EMA_12']=float(df1.at[0,'收盤價'])
df1.loc[0,'EMA_26']=float(df1.at[0,'收盤價'])
df1.loc[0,'MACD']=0
for i in range(1,len(df1)):
    A=EMA(i,12)
    B=EMA(i,26)
    dif=A-B
    macd=MACD(i,14,dif)
    df1.loc[i,'EMA_12']=A
    df1.loc[i,'EMA_26']=B
    df1.loc[i,'MACD']=macd
#psy心理線
'''
PSY = 有上漲的天數( N日內 ) / N * 100
上漲天數係指週期天數( N日 )內，股價上漲的天數和由於其一定在 0 ~ 100之間移動
故研判PSY的數值當線路介於 20 ~ 80之間移動時為盤整狀態；
當數值低於10時則可能出現反彈機會，應注意買點的出現；
數值若高於90以上時，則可能短線過熱，市場心理過於超買，極可能出現回檔的現象。
PSY的應用
1. 一般心理線介於25%~75%是合理變動範圍。 
2. 超過75%或低於25%，就有買超或賣超現象，股價回跌或回升機會增加，此時可準備賣出或買進。
在大多頭或大空頭市場初期，可將超買、超賣點調整至83%、17%值到行情尾聲，再調回70%、25%。
3. 當行情出現低於10%或高於90%時；是真正的超賣和超買現象，
行情反轉的機會相對提高，此時為賣出和買進時機
'''
def psy(index,N):
    riseday=0
    for i in range(index-N,index):
        isrise=(float(df1.at[i,'漲跌價差']))
        if (isrise>0):
            riseday+=1
    ans=riseday/N*100
    return ans
#psy_6
for i in range(0,6):
    df1.loc[i,'psy_6']=0
for i in range(6,len(df1)):
    df1.loc[i,'psy_6']=psy(i,6)
#MTM動量指標
'''
MTM＝C－Cn
其中：C為當日收市價，Cn為N日前收市價，N為設定參數，一般選設10日，亦可在6日至14日之間選擇。
（1）一般情況下，MTM由上向下跌破中心線時為賣出時機，相反，MTM由下向上突破中心線時為買進時機。
（2）因選設10日移動平均線情況下，當MTM在中心線以上，由上向下跌穿平均為賣出訊號，反之，當MTM在中心線以下，由下向上突破平均線為買入訊號。
（3）股價在上漲行情中創新高點，而MTM未能配合上升，出現背馳現象，意味上漲動力減弱，此時應關註行情，慎防股價反轉下跌。
（4）股價在下跌行情中走出新低點，而MTM未能配合下降，出現背馳，該情況意味下跌動力減弱，此時應註意逢低承接。
（5）若股價與MTM在低位同步上升，顯示短期將有反彈行情；若股價與MTM在高位同步下降，則顯示短期可能出現股價回落。
'''
def MTM(index,N):
    C=(float(df1.at[i,'收盤價']))
    Cn=(float(df1.at[i-N,'收盤價']))
    return C-Cn
#MTM_6
for i in range(0,6):
    df1.loc[i,'MTM_6']=0
for i in range(6,len(df1)):
    df1.loc[i,'MTM_6']=MTM(i,6)
# SAR 停損點轉向操作系統
'''
inti：
    上升波段：SAR 設定於近期n日中最高價
    下跌波段：SAR 設定於近期n日中最低價

SAR：
    當天SAR=前一天SAR+AF*(區間極值–前一天SAR)
    
AF：
    則是SAR分析指標的特有產物，叫作加速因子，起始值為0.02，
    當趨勢正在走上漲(下跌)波段的時候，只要最高(低)價再創新高(低)，AF就增加0.02，而最高限制為0.2。
    
區間極值：
    上漲的波段當中，取最高價當作區間極值；相反的，在下跌波段當中，則取最低價當作區間極值。
'''

AF = 0.02
n_day=6
SAR_col_name = 'SAR_%d' % n_day

def SAR(index, n_day):
    
    global AF

    if index < n_day:  # for init
        return 0
    flut_cumu = sum(df1.loc[index-n_day:index-1,'漲跌價差'])
    
    # 算區間極值
    if flut_cumu >= 0: # 上漲
        extrema = float(max(df1.loc[index-n_day:index-1, "最高價"]))
    else:  # 下跌
        extrema = float(min(df1.loc[index-n_day:index-1, "最低價"]))
        
    # 更新 AF值
    if index > n_day+1:
         # 上次是上漲且上漲創新高 或 上次是下跌且下跌創新低
        if (SAR.last_up_down == 0 and flut_cumu < 0 and SAR.last_extrema > extrema) or (SAR.last_up_down == 1 and flut_cumu >= 0 and SAR.last_extrema < extrema):
            AF = 0.2 if AF>=0.2 else AF+0.02     
    
    # 保存漲跌資訊到下一次
    if flut_cumu >= 0: # 上漲
        SAR.last_up_down = 1
    else:  # 下跌
        SAR.last_up_down= 0
    
    # 計算 SAR
    if index == n_day:  # for init
        SAR.last_extrema = extrema
        return extrema
    else:
        last_SAR = float(df1.loc[index-1, SAR_col_name])
        return last_SAR+AF*(extrema - last_SAR)


for i in range(0,len(df1)):
    df1.loc[i, SAR_col_name] = SAR(i,n_day)

#  DMI 趨向指標
'''
一. 先計算股票價位的變動值；分別以DM+ 或 DM- 來代表其上漲或下跌的趨向變動值DM：
  
 　　　DM+ = 本日最高價 - 一日前最高價 ( 上漲的趨向變動值 )
 　　　DM- = 本日最低價 - 一日前最低價 ( 下跌的趨向變動值 )
  
 二. 無論其DM+ 或DM- 接取其絕對值較大之數值為當日之趨向變動值。由此原則可得此趨向之變動值在於求取每日價格波動幅度之增減的真正幅度。
  
 三. 找出TR，該變動值需比較下列三種差價的『絕對值』後，取其中最大者為本日之TR。
  
 　　　A = 今日最高價 - 今日最低價
 　　　B = 今日最高價 - 一日前收盤價
 　　　C = 今日最低價 - 一日前收盤價
    
  
 四. 計算DI：
  
 　　　+DI = (DM+) 14日平均 / (TR) 14日平均 *100
 　　　-DI = (DM-) 14日平均 / (TR) 14日平均 *100
 　　　求出其14日移動平均值
 　　　第一日：採用+DI及-DI的14日移動平均值
 　　　第二日：開始以平滑移動方式修正：
 　　　本日的(+DI)十日平均 = 一日前(DI+)值 * 13/14 + 本日(+DI) * 1/14
 　　　本日的(-DI)十日平均 = 一日前(DI-)值 * 13/14 + 本日(-DI) * 1/14
  
 五. ADX計算方式：
 　　　ADX= [(+DI)- (-DI)] / [(+DI)+(-DI)] *100
 　　　求出ADX其10日移動平均值
 　　　第一日：直接採用ADX的14日移動平均值
 　　　第二日：開始以平滑移動方式修正：
 　　　ADX= (一日前的ADX * 13/14) + (本日的ADX * 1/14)
'''

n_day = 14

def DM(index):
    if index==0:
        return (0,0)
    DMP = float(df1.at[index, "最高價"]) - float(df1.at[index-1, "最高價"])
    DMN = float(df1.at[index, "最低價"]) - float(df1.at[index-1, "最低價"])
    return (DMP, DMN)

def TR(index): 
    if index==0:
        return 0
    A = abs(float(df1.at[index, "最高價"]) - float(df1.at[index, "最低價"]))
    B = abs(float(df1.at[index, "最高價"]) - float(df1.at[index-1, "收盤價"]))
    C = abs(float(df1.at[index, "最低價"]) - float(df1.at[index-1, "收盤價"]))
    
    return max([A, B, C])

def DI(index, n_day):

    if index < n_day:
        return (0, 0)
    elif index == n_day:
        PDI = float(sum(df1.loc[index-n_day:index-1, "DM+(DMI)"]))/float(sum(df1.loc[index-n_day:index-1, "TR(DMI)"]))*100
        NDI = float(sum(df1.loc[index-n_day:index-1, "DM-(DMI)"]))/float(sum(df1.loc[index-n_day:index-1, "TR(DMI)"]))*100
      
        return (PDI, NDI)
    else:
        PDI = float(sum(df1.loc[index-n_day:index-1, "DM+(DMI)"]))/float(sum(df1.loc[index-n_day:index-1, "TR(DMI)"]))*100
        NDI = float(sum(df1.loc[index-n_day:index-1, "DM-(DMI)"]))/float(sum(df1.loc[index-n_day:index-1, "TR(DMI)"]))*100
        
        return (float(df1.loc[index-1, "+DI(DMI)"])*(1-1/n_day) + PDI/n_day, float(df1.loc[index-1, "-DI(DMI)"])*(1-1/n_day) + NDI/n_day)
    
    
def ADX(index, n_day):

    # ADX= [(+DI)- (-DI)] / [(+DI)+(-DI)] *100
    
    if index < n_day:
        return 0
    elif index == n_day:
        PDI = float(df1.loc[index, "+DI(DMI)"])
        NDI = float(df1.loc[index, "-DI(DMI)"])
        ADX = (PDI - NDI) / (PDI + NDI) * 100
        return ADX
    else:
        PDI = float(df1.loc[index, "+DI(DMI)"])
        NDI = float(df1.loc[index, "-DI(DMI)"])
        ADX = float(df1.loc[index-1, "ADX(DMI)"])*(1-1/n_day) + ((PDI - NDI)/(PDI + NDI)*100)/n_day
        
        return ADX

for i in range(0,len(df1)):
    (df1.loc[i, "DM+(DMI)"], df1.loc[i, "DM-(DMI)"]) = DM(i)
    df1.loc[i, "TR(DMI)"] = TR(i)
    (df1.loc[i, "+DI(DMI)"], df1.loc[i, "-DI(DMI)"]) = DI(i, n_day)
    df1.loc[i, "ADX(DMI)"] = ADX(i, n_day)

#CDP逆勢操作系統
'''
1.  先求出昨日行情的CDP值(亦稱均價)
    CDP = (最高價 + 最低價 + 2*收盤價) /4
    
2.  再分別計算昨天行情得最高值(AH)、近高值(NH)、近低值(NL)及最低值(AL)
    AH = CDP + (最高價 - 最低價)
    NH = 2*CDP - 最低價
    NL = 2*CDP - 最高價
    AL = CDP - (最高價 - 最低價
    
3.  以最高值(AH)附近開盤應追價買進
    盤中高於近高值(NH)時可以賣出
    盤中低於近低值(NL)時可以買進
    以最低值(AL)附近開盤應追價賣出
    CDP為當天軋平的超短線操作法，務必當天沖銷(利用融資融卷)軋平。若當天盤中無法達到所設定理想的買賣價位時，亦應以當日的收盤價軋平    

'''

def CDP(index) :
    ans = (float(df1.at[index-1,'最高價']) + float(df1.at[index-1,'最低價']) + (float(df1.at[index-1,'收盤價'])*2)) / 4
    return ans

df1.loc[0,'CDP']=np.nan
df1.loc[0,'AH']=np.nan
df1.loc[0,'NH']=np.nan
df1.loc[0,'NL']=np.nan
df1.loc[0,'AL']=np.nan

for i in range(1,len(df1)):
    cdp = CDP(i)
    df1.loc[i,'CDP'] = cdp
    ah = float(df1.at[i,'CDP']) + ( float(df1.at[i-1,'最高價']) - float(df1.at[i-1,'最低價']) )
    nh = ( float(df1.at[i,'CDP']) * 2 ) - float(df1.at[i-1,'最低價'])
    nl = ( float(df1.at[i,'CDP']) * 2 ) - float(df1.at[i-1,'最高價'])
    al = float(df1.at[i,'CDP']) - ( float(df1.at[i-1,'最高價']) - float(df1.at[i-1,'最低價']) )

    df1.loc[i,'AH'] = ah
    df1.loc[i,'NH'] = nh
    df1.loc[i,'NL'] = nl
    df1.loc[i,'AL'] = al
    
    st = float(df1.at[i,'開盤價'])
    
    if st >= ah :
        df1.loc[i,'Trend'] = 6
    elif st >= nh :
        df1.loc[i,'Trend'] = 5
    elif st >= cdp :
        df1.loc[i,'Trend'] = 4
    elif st >= nl :
        df1.loc[i,'Trend'] = 3
    elif st >= al :
        df1.loc[i,'Trend'] = 2
    else :
        df1.loc[i,'Trend'] = 1
        
'''
6 = 大漲
5 = 漲
4 = 偏漲
3 = 偏跌
2 = 跌
1 = 大跌
'''
def KDX(index) :
    ky1 = float(df1.at[index-1,'K'])
    ky2 = float(df1.at[index,'K'])
    dy1 = float(df1.at[index-1,'D'])
    dy2 = float(df1.at[index,'D'])
    
    k1 = ky2 - ky1
    b1 = ky1 - k1
    k2 = dy2 - dy1
    b2 = dy1 - k2
    
    x = (b2 - b1) / (k1 - k2)
    y = (k1*x) + b1
    return y
    
N_day=5

for i in range(N_day,len(df1)):
    Yesterday=df1.loc[i-1,'D']-df1.loc[i-1,'K']
    Today    =df1.loc[i,'D']-df1.loc[i,'K']
    Product = Yesterday*Today
    
    if Product < 0 or (abs(Product-0)<0.0001) :
        if Yesterday > 0:
            a=1
        else :
            a=-1
    else :
        a=0        
df1=df1.fillna(0)
df1=df1.drop(range(14))
df1=df1.reset_index(drop=True)
df1

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數,RSI_6,...,TR(DMI),+DI(DMI),-DI(DMI),ADX(DMI),CDP,AH,NH,NL,AL,Trend
0,99/01/22,29477562,1612081382,55.20,55.20,54.35,54.40,-1.55,3929,8.860759,...,1.60,-2.336449,-2.803738,-9.090909,56.0125,56.6625,56.275,55.625,55.3625,1.0
1,99/01/25,12845252,695785374,53.95,54.45,53.80,54.20,-0.20,2147,0.000000,...,0.65,-3.011604,-3.590695,-9.068063,54.5875,55.4375,54.825,53.975,53.7375,2.0
2,99/01/26,26395774,1416284559,54.15,54.55,52.75,52.95,-1.25,4558,0.000000,...,1.80,-4.230915,-4.710095,-8.803155,54.1625,54.8125,54.525,53.875,53.5125,3.0
3,99/01/27,14855839,787198926,52.95,53.30,52.70,52.90,-0.05,2182,23.000000,...,0.60,-5.721424,-6.474500,-8.615417,53.3000,55.1000,53.850,52.050,51.5000,3.0
4,99/01/28,12622891,676089714,53.10,54.20,53.10,54.05,1.15,1822,20.175439,...,1.30,-8.084180,-8.640607,-8.237670,52.9500,53.5500,53.200,52.600,52.3500,4.0
5,99/01/29,13799786,727944433,52.95,53.25,52.20,53.00,-1.05,2804,19.658120,...,1.85,-9.442398,-10.177184,-7.916777,53.8500,54.9500,54.600,53.500,52.7500,2.0
6,99/02/01,19637117,1030361997,53.15,53.20,51.95,52.40,-0.60,2590,23.711340,...,1.25,-11.216920,-11.950242,-7.577389,52.8625,53.9125,53.525,52.475,51.8125,4.0
7,99/02/02,11987878,625747966,52.95,52.95,51.60,51.85,-0.55,2350,36.936937,...,1.35,-12.690663,-13.689608,-7.306627,52.4875,53.7375,53.025,51.775,51.2375,4.0
8,99/02/03,13598236,714184048,52.10,52.80,52.05,52.75,0.90,1744,45.555556,...,0.95,-13.777543,-15.132282,-7.119445,52.0625,53.4125,52.525,51.175,50.7125,4.0
9,99/02/04,5940528,312541875,52.75,52.85,52.35,52.55,-0.20,982,32.800000,...,0.50,-15.056516,-16.290914,-6.892185,52.5875,53.3375,53.125,52.375,51.8375,4.0


In [52]:
df2=df1[['RSI_6','RSI_12','K','D','BIAS','WMR','EMA_12','EMA_26','MACD','psy_6','MTM_6','SAR_6','DM+(DMI)','DM-(DMI)','TR(DMI)','+DI(DMI)','-DI(DMI)','ADX(DMI)','Trend']].copy()
new_idx=pd.DataFrame()
#rsi
rsi6=np.array([float(i) for i in df2['RSI_6']])
rsi12=np.array([float(i) for i in df2['RSI_12']])
rst=np.array(0)
for i in range(1,rsi6.size):
    tmp=0
    if(rsi6[i]>90):
        tmp=-1
    elif(rsi6[i]>80):
        tmp=-0.5
    elif(rsi6[i]<10):
        tmp=1
    elif(rsi6[i]<20):
        tmp=0.5
    dif=rsi6[i]-rsi12[i]
    mul=(rsi6[i-1]-rsi12[i-1])*dif
    if (abs(mul-0) < 0.0000001) or (mul < 0):
        if (dif > 0) or ((abs(dif-0) < 0.0000001) and rsi6[i-1] < rsi12[i-1]):
            tmp=tmp+2
        else:
            tmp=tmp-2
    rst=np.append(rst,tmp)
new_idx['RSI']=np.reshape(rst, (-1)).tolist()
#KD
k=np.array([float(i) for i in df2['K']])
d=np.array([float(i) for i in df2['D']])
rst=np.array(0)
for i in range(1,k.size):
    dif=k[i]-d[i]
    mul=dif*(k[i-1]*d[i-1])
    tmp=0
    if (abs(mul-0) < 0.0000001) or (mul < 0):
        if (dif > 0) or ((abs(dif-0) < 0.0000001) and k[i-1] < d[i-1]):
            tmp=1
        else:
            tmp=-1
    w=0
    if k[i]>80 or k[i]<20 :
        w=2
    elif k[i]>70 or k[i]<30 :
        w=1
    rst=np.append(rst,w*tmp)
new_idx['KD']=np.reshape(rst, (-1)).tolist()
#BIAS
bias=np.array([float(i) for i in df2['BIAS']])
if(bias[0]<-4.5):
    rst=np.array(2)
elif(bias[0]<-3):
    rst=np.array(1)
elif(bias[0]>5):
    rst=np.array(-2)
elif(bias[0]>3.5):
    rst=np.array(-1)
else:
    rst=np.array(0)
for i in range(1,bias.size):
    if(bias[i]<-4.5):
        rst=np.append(rst,2)
    elif(bias[i]<-3):
        rst=np.append(rst,1)
    elif(bias[i]>5):
        rst=np.append(rst,-2)
    elif(bias[i]>3.5):
        rst=np.append(rst,-1)
    else:
        rst=np.append(rst,0)
new_idx['BIAS']=np.reshape(rst, (-1)).tolist()
#WMR
wr=np.array([float(i) for i in df2['WMR']])
if(wr[0]<=20):
    rst=np.array(-2)
elif(wr[0]<35):
    rst=np.array(-1)
elif(wr[0]<65):
    rst=np.array(0)
elif(wr[0]<80):
    rst=np.array(1)
else:
    rst=np.array(2)
for i in range(1,wr.size):
    if(wr[i]<=20):
        rst=np.append(rst,-2)
    elif(wr[i]<35):
        rst=np.append(rst,-1)
    elif(wr[i]<65):
        rst=np.append(rst,0)
    elif(wr[i]<80):
        rst=np.append(rst,1)
    else:
        rst=np.append(rst,2)
new_idx['WMR']=np.reshape(rst, (-1)).tolist()
#MACD
macd=np.array([float(i) for i in df2['MACD']])
dif=np.array([float(i) for i in df2['EMA_12']])-np.array([float(i) for i in df2['EMA_26']])
if macd[0]>0 and dif[0]>0:
    rst=np.array(2)
elif dif[0]>0:
    rst=np.array(1)
elif macd[0]>0:
    rst=np.array(-1)
else:
    rst=np.array(-2)
for i in range(1,macd.size):
    if macd[i]>0 and dif[i]>0:
        rst=np.append(rst,2)
    elif dif[i]>0:
        rst=np.append(rst,1)
    elif macd[i]>0:
        rst=np.append(rst,-1)
    else:
        rst=np.append(rst,-2)
new_idx['MACD']=np.reshape(rst, (-1)).tolist()
#PSY
new_idx['PSY']=df2['psy_6'].copy()
#SAR
sar=np.array([float(i) for i in df2['SAR_6']])
dif=k-sar
rst=np.array(0)
for i in range(1,k.size):
    mul=dif[i]*dif[i-1]
    tmp=0
    if (abs(mul-0) < 0.0000001) or (mul < 0):
        if (dif[i] > 0) or ((abs(dif[i]-0) < 0.0000001) and dif[i-1]<0):
            tmp=1
        else:
            tmp=-1
    rst=np.append(rst,tmp)
new_idx['SAR']=np.reshape(rst, (-1)).tolist()
#CDP
new_idx['CDP']=df2['Trend'].copy()
#DMI
l=['+DI(DMI)','-DI(DMI)','ADX(DMI)']
for i in l:
    new_idx[i]=df2[i].copy()
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
print(bias)
new_idx

[-3.1043257  -2.25428314 -3.36331639 ... -1.39806569  0.52621508
  1.25167635]


,RSI,KD,BIAS,WMR,MACD,PSY,SAR,CDP,+DI(DMI),-DI(DMI),ADX(DMI)
0,0.0,0,1,2,-1,33.333333,0,1.0,-2.336449,-2.803738,-9.090909
1,1.0,0,0,1,-1,16.666667,0,2.0,-3.011604,-3.590695,-9.068063
2,1.0,-1,1,2,-2,0.000000,0,3.0,-4.230915,-4.710095,-8.803155
3,0.0,-2,0,2,-2,0.000000,0,3.0,-5.721424,-6.474500,-8.615417
4,0.0,0,0,0,-2,0.000000,0,4.0,-8.084180,-8.640607,-8.237670
5,0.5,0,0,1,-2,16.666667,0,2.0,-9.442398,-10.177184,-7.916777
6,2.0,0,0,1,-2,16.666667,0,4.0,-11.216920,-11.950242,-7.577389
7,0.0,-1,0,2,-2,16.666667,0,4.0,-12.690663,-13.689608,-7.306627
8,0.0,-1,0,0,-2,16.666667,0,4.0,-13.777543,-15.132282,-7.119445
9,0.0,0,0,0,-2,33.333333,0,4.0,-15.056516,-16.290914,-6.892185


In [55]:
#Normalize

def min_max(inputCol):
    arr=np.array( [float(i) for i in df1[inputCol]])
    arr = np.reshape(arr, (-1,1))
    min_max_scaler=preprocessing.MinMaxScaler()
    a=min_max_scaler.fit_transform(arr)              
    min_maxdf[inputCol]=np.reshape(a*255, (-1)).tolist()
min_maxdf = pd.DataFrame()
for i in ['RSI_6','RSI_12','K','D','BIAS','WMR','EMA_12','EMA_26','MACD','psy_6','MTM_6','SAR_6','DM+(DMI)','DM-(DMI)','TR(DMI)','+DI(DMI)','-DI(DMI)','ADX(DMI)','Trend']:
    min_max(i)
min_maxdf

,RSI_6,RSI_12,K,D,BIAS,WMR,EMA_12,EMA_26,MACD,psy_6,MTM_6,SAR_6,DM+(DMI),DM-(DMI),TR(DMI),+DI(DMI),-DI(DMI),ADX(DMI),Trend
0,22.843233,61.076545,64.125806,96.841275,83.322777,218.132530,55.864922,53.097138,162.926848,85.0,110.697674,64.252685,90.913043,122.218935,64.304348,92.915677,99.573097,192.224406,0.0
1,0.000000,51.714171,65.419346,80.827073,105.746251,180.303030,53.506379,51.837095,157.966518,42.5,99.825581,62.084673,110.869565,147.869822,22.173913,89.916112,96.102739,192.234719,51.0
2,0.000000,41.076625,33.830771,57.217826,76.490801,242.250000,50.240558,50.037450,151.136844,0.0,81.046512,59.320996,148.565217,132.781065,73.173913,84.498979,91.166361,192.354299,102.0
3,59.294505,68.124408,11.297387,32.252219,92.138883,243.000000,47.426365,48.345795,143.262640,0.0,94.883721,55.543215,88.695652,162.958580,19.956522,77.876976,83.385612,192.439045,102.0
4,52.012724,62.182092,75.340520,41.827346,171.502442,142.678571,46.213646,47.361751,136.004216,0.0,124.534884,51.779017,184.043478,176.538462,51.000000,67.379774,73.833419,192.609561,153.0
5,50.679065,64.049677,73.348519,47.395545,144.752560,172.615385,44.120589,45.918931,128.555113,42.5,112.674419,48.221850,102.000000,137.307692,75.391304,61.345509,67.057360,192.754413,51.0
6,61.128356,43.456887,63.426307,47.045641,132.891117,185.847458,41.739878,44.279175,120.779373,42.5,131.453488,44.341144,141.913043,156.923077,48.782609,53.461695,59.238451,192.907613,153.0
7,95.224235,56.726851,31.028730,33.548570,115.648936,233.389831,39.166573,42.482389,112.652811,42.5,124.534884,40.574269,133.043478,153.905325,53.217391,46.914182,51.568118,193.029836,153.0
8,117.443223,57.388843,51.618709,32.979752,163.427726,142.211538,37.903655,41.274415,105.451060,42.5,167.034884,37.455400,137.478261,178.047337,35.478261,42.085415,45.206152,193.114330,153.0
9,84.559121,51.091730,105.660404,54.725560,168.802996,113.716216,36.631811,40.054650,99.054899,85.0,164.069767,34.960305,146.347826,173.520710,15.521739,36.403219,40.096765,193.216916,153.0


In [56]:
def min_max2(inputCol):
    arr=np.array( [float(i) for i in new_idx[inputCol]])
    arr = np.reshape(arr, (-1,1))
    min_max_scaler=preprocessing.MinMaxScaler()
    a=min_max_scaler.fit_transform(arr)              
    min_maxdf2[inputCol]=np.reshape(a*255, (-1)).tolist()
min_maxdf2 = pd.DataFrame()
for i in ['RSI','KD','BIAS','WMR','MACD','PSY','SAR','CDP','+DI(DMI)','-DI(DMI)','ADX(DMI)']:
    min_max2(i)
min_maxdf2

,RSI,KD,BIAS,WMR,MACD,PSY,SAR,CDP,+DI(DMI),-DI(DMI),ADX(DMI)
0,139.090909,255.0,127.5,255.00,63.75,85.0,127.5,0.0,92.915677,99.573097,192.224406
1,185.454545,255.0,0.0,191.25,63.75,42.5,127.5,51.0,89.916112,96.102739,192.234719
2,185.454545,127.5,127.5,255.00,0.00,0.0,127.5,102.0,84.498979,91.166361,192.354299
3,139.090909,0.0,0.0,255.00,0.00,0.0,127.5,102.0,77.876976,83.385612,192.439045
4,139.090909,255.0,0.0,127.50,0.00,0.0,127.5,153.0,67.379774,73.833419,192.609561
5,162.272727,255.0,0.0,191.25,0.00,42.5,127.5,51.0,61.345509,67.057360,192.754413
6,231.818182,255.0,0.0,191.25,0.00,42.5,127.5,153.0,53.461695,59.238451,192.907613
7,139.090909,127.5,0.0,255.00,0.00,42.5,127.5,153.0,46.914182,51.568118,193.029836
8,139.090909,127.5,0.0,127.50,0.00,42.5,127.5,153.0,42.085415,45.206152,193.114330
9,139.090909,255.0,0.0,127.50,0.00,85.0,127.5,153.0,36.403219,40.096765,193.216916


In [2]:
#LABEL GENERATE
List_Low=np.array(df1['最低價'])
List_Hig=np.array(df1['最高價'])
List_price=np.array(df1['收盤價'])
List_ans=np.array(0)
for i in range(19,List_price.size-1):
    LL=float(List_Low[i-1])
    LH=float(List_Hig[i-1])
    CL=float(List_Low[i])
    CH=float(List_Hig[i])
    RL=float(List_Low[i+1])
    RH=float(List_Hig[i+1])
    LP=float(List_price[i-1])
    CP=float(List_price[i])
    RP=float(List_price[i+1])
    cnt=2
    while(CP==LP):
        LP=float(List_price[i-cnt])
        cnt=cnt+1
    cnt=2
    while(CP==RP):
        RP=float(List_price[i+cnt])
        cnt=cnt+1
    if LP>CP and RP>CP:
        List_ans=np.append(List_ans,1)
    elif CP>LP and CP>RP:
        List_ans=np.append(List_ans,-1)
    else:
        List_ans=np.append(List_ans,0)
    '''
    if LL>CH and RL>CH :
        List_ans=np.append(List_ans,1)
    elif CL>=LH and CL>RH:
        List_ans=np.append(List_ans,-1)
    else:
        List_ans=np.append(List_ans,0)
    '''
List_ans=np.append(List_ans,0)
import sys
np.set_printoptions(threshold=sys.maxsize)
print(List_ans)

[ 0  1 -1  0  0  1  0  0 -1 -1  0  0  1  0  0 -1  0  0  1  0  0  0 -1  0
  1  0  0  0 -1  1  0 -1  1  0 -1  0  1  0 -1  1  0 -1  0  0  0  0  1 -1
  0  1  1 -1  0  1 -1  0  1 -1  0  0  1 -1  1  0  0  0 -1  0  1 -1  0  0
  0  1  0  0  0  0 -1  1 -1  0  1 -1  1 -1  0  0  1  0  0 -1  1  0 -1  0
  1  0 -1  0  1 -1  1  0  0 -1  1  0 -1  1  0 -1  0  1  0 -1  0  0  1  0
 -1  0  1  0  0 -1  0  0  1  0 -1  1  0  0  0 -1  0  1  1  0  0  0 -1  1
 -1  1 -1 -1  1 -1  1 -1  1 -1 -1  1  0 -1  0  0  1  0 -1  0  0  1  0  0
 -1  1 -1  1 -1  1 -1 -1  1  0 -1  1 -1  0  0  0  1 -1  1  0  0 -1  0  1
 -1  1  0  0  0  0  0 -1  1  1 -1  1 -1  1  0  0 -1  1  0  0 -1  1 -1  1
  0  0  0 -1  0  0  0  1  0  0 -1  0  0  1  0 -1  0  0  1  0  0  0 -1  0
  0  0  1  0  0 -1  1 -1 -1  0  1  1  0 -1  1  0 -1  1 -1  0  0  0  0  0
  1  1  0  0  0  0  0 -1  1  0 -1  1  0  0 -1  0  0  1 -1  0  0  0  1  0
 -1  0  0  1 -1 -1  1 -1  1 -1  1 -1 -1  1 -1  0  0  1 -1  1 -1  1 -1  1
 -1  0  1  0 -1  1 -1 -1  0  0  0  1 -1  0  0  0  1

In [34]:
#generate img
from PIL import Image
v=min_maxdf.to_numpy()
v=v.astype(np.float32)
for i in range(len(min_maxdf)-19):
    img=Image.fromarray(v[i:i+19,:],'F')
    date=str(df1['日期'][i+19]).replace("/","")
    PATH='C:/Users/user/NEW/56/'+str(List_ans[i+1])+'/'+date+'.jpg'
    PATH=PATH.replace(" ","")
    print(PATH)
    img = img.convert('RGB')
    img.save(PATH,"JPEG")

C:/Users/user/NEW/56/1/990226.jpg
C:/Users/user/NEW/56/-1/990301.jpg
C:/Users/user/NEW/56/0/990302.jpg
C:/Users/user/NEW/56/0/990303.jpg
C:/Users/user/NEW/56/1/990304.jpg
C:/Users/user/NEW/56/0/990305.jpg
C:/Users/user/NEW/56/0/990308.jpg
C:/Users/user/NEW/56/-1/990309.jpg
C:/Users/user/NEW/56/-1/990310.jpg
C:/Users/user/NEW/56/0/990311.jpg
C:/Users/user/NEW/56/0/990312.jpg
C:/Users/user/NEW/56/1/990315.jpg
C:/Users/user/NEW/56/0/990316.jpg
C:/Users/user/NEW/56/0/990317.jpg
C:/Users/user/NEW/56/-1/990318.jpg
C:/Users/user/NEW/56/0/990319.jpg
C:/Users/user/NEW/56/0/990322.jpg
C:/Users/user/NEW/56/1/990323.jpg
C:/Users/user/NEW/56/0/990324.jpg
C:/Users/user/NEW/56/0/990325.jpg
C:/Users/user/NEW/56/0/990326.jpg
C:/Users/user/NEW/56/-1/990329.jpg
C:/Users/user/NEW/56/0/990330.jpg
C:/Users/user/NEW/56/1/990331.jpg
C:/Users/user/NEW/56/0/990401.jpg
C:/Users/user/NEW/56/0/990402.jpg
C:/Users/user/NEW/56/0/990406.jpg
C:/Users/user/NEW/56/-1/990407.jpg
C:/Users/user/NEW/56/1/990408.jpg
C:/Users